## bs4解析
 - 为什么需要在爬虫中使用数据解析
    - 就是为了可以实现聚焦爬虫
 - 数据解析的通用原理是什么？（解析的数据只会存在于标签之间或者属性中）
    - html 是用来展示数据的
    - 数据的提取
 - bs4数据解析的原理/流程
    - 实例化一个BeautifulSoup的对象，且将等待被解析的数据加载到该对象中
        - 方法一：
            - BeautifulSoup(fp,'lxml'):解析本地储存的html文件
        - 方法二：
            - BeautifulSoup(page_text,'lxml'):解析会联网上请求到的页面数据
    - 调用BeautifulSoup对象中相关方法和属性进行标签的定位和数据的提取
 - 环境的安装
    - pip install bs4
    - pip install lxml


- 标签定位
    - soup.tagNmae：返回第一次出现的tagName的标签
    - 属性定位：soup.find('tagNmae',attrName='Value')
    - findall和find的用法一样，但是返回值不同
    - 选择器定位：
        - select('selector')

In [48]:
import os.path
import time

from bs4 import BeautifulSoup

In [49]:
#bs4中有哪些方法和属性可以被我们使用
fp=open('./人民.html','r',encoding='utf-8')
soup=BeautifulSoup(fp,'lxml')
print(soup)

<!DOCTYPE HTML>
<html>
<head>
<meta charset="utf-8"/>
<link href="//www.sogou.com/images/logo/new/favicon.ico?v=4" rel="shortcut icon" type="image/x-icon"/>
<title>搜狗搜索</title>
<link href="static/css/anti.min.css?v=1" rel="stylesheet"/>
<script src="//dlweb.sogoucdn.com/common/lib/jquery/jquery-1.11.0.min.js"></script>
<script src="static/js/antispider.min.js?v=3"></script>
<script>

        var domain = getDomain();

        window.imgCode = -1;



        (function() {

            function checkSNUID() {

                var cookieArr = document.cookie.split('; '),

                    count = 0;



                for(var i = 0, len = cookieArr.length; i < len; i++) {

                    if (cookieArr[i].indexOf('SNUID=') > -1) {

                        count++;

                    }

                }



                return count > 1;

            }



            if(checkSNUID()) {

                var date = new Date(), expires;

                date.setTime(date.getTime()

In [50]:
#标签的定位
soup.div

<div class="header">
<div class="logo">
<a href="/">
<img height="60" src="static/images/logo_180x60.png" srcset="static/images/logo_180x60@2x.png 2x" width="180"/>
</a>
</div>
<div class="other"><span class="s1">您的访问出错了</span><span class="s2"><a href="/">返回首页&gt;&gt;</a></span></div>
</div>

In [51]:
#属性的定位 根据属性定位具体的标签
soup.find('div',class_='')
soup.find('a',id='')

<a href="/">
<img height="60" src="static/images/logo_180x60.png" srcset="static/images/logo_180x60@2x.png 2x" width="180"/>
</a>

In [52]:
soup.findAll('a',class_='g')

[<a class="g" href="http://www.miibeian.gov.cn" target="_blank">京ICP证050897号</a>]

In [53]:
soup.select('#sxl')#根据id选择器定位a标签

[<p class="p5" id="sxl">
 <a href="javascript:void(0);" id="submit">提交</a>
 <span>提交后没解决问题？欢迎<a href="http://fankui.help.sogou.com/index.php/web/web/index?type=10&amp;anti_time=1680661116&amp;domain=www.sogou.com" target="_blank">反馈</a>。</span>
 <!--span>提交后没解决问题？欢迎<a href="http://fankui.help.sogou.com/index.php/web/web/index?type=10&anti_time=1680661116&domain=www.sogou.com&verifycode=cb7267d3c167" target="_blank">反馈</a>。</span-->
 </p>]

In [54]:
soup.select('.g')#定位class为g的标签

[<a class="g" href="http://www.miibeian.gov.cn" target="_blank">京ICP证050897号</a>]

层级选择器

In [55]:
soup.select('#sxl > a ')

[<a href="javascript:void(0);" id="submit">提交</a>]

# 数据解析
    - 正则
    - bs4
    - xpath()重点
    - pyquery（小众）

- 数据的提取
    - 提取标签中的存在的数据
        - .string
        - .text
    - 提取标签属性中存储的数据
        - tagName['attrName']

In [56]:
soup.div

<div class="header">
<div class="logo">
<a href="/">
<img height="60" src="static/images/logo_180x60.png" srcset="static/images/logo_180x60@2x.png 2x" width="180"/>
</a>
</div>
<div class="other"><span class="s1">您的访问出错了</span><span class="s2"><a href="/">返回首页&gt;&gt;</a></span></div>
</div>

In [57]:
soup.div.text

'\n\n\n\n\n\n您的访问出错了返回首页>>\n'

In [58]:
soup.div

<div class="header">
<div class="logo">
<a href="/">
<img height="60" src="static/images/logo_180x60.png" srcset="static/images/logo_180x60@2x.png 2x" width="180"/>
</a>
</div>
<div class="other"><span class="s1">您的访问出错了</span><span class="s2"><a href="/">返回首页&gt;&gt;</a></span></div>
</div>

In [59]:
soup.div.string

In [60]:
soup.div['class']

['header']

使用bs4解析爬取三国演义的整片小说的内容：https://www.shicimingju.com/book/sanguoyanyi.html

流程分析：
1. 获取页面
2. 分析页面
- 获取url和章节名称
- 根据url和章节名称获取正文内容


In [63]:
## 1. 获取首页

import requests
url = 'https://www.shicimingju.com/book/sanguoyanyi.html'
header={
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}

page_index_text=requests.get(url=url,headers=header)
page_index_text=page_index_text.text

In [ ]:
#获取章节名称和对应的url
import time
soup=BeautifulSoup(page_index_text,'lxml')
a_list=soup.select('.book-mulu > ul > li > a')
with open('./sgyy.txt','w',encoding='utf-8')as f:
    for a in a_list:
        chapter_href='https://www.shicimingju.com/'+a['href']
        chapter_title=a.text
        # print('chapter_title:{},herf:{}'.format(chapter_title,chapter_href))
        chapter_content=requests.get(chapter_href,headers=header)
        chapter_content.encoding='utf-8'
        chapter_content=chapter_content.text
        # print(chapter_content)
        soup=BeautifulSoup(chapter_content,'lxml')
        chapter_text=soup.find('div',class_='chapter_content').text
        f.write('{}\n{}\n'.format(chapter_title,chapter_text))
print('over!!!!')

### xpath解析

- html 标签
    - 是一个树状的结构
- xpath 解析原理
    - 实例化一个etree对象，并且将被解析的数据加载到该对象中
         - 解析本地储存的html对象
            - etree.parase('fileName')
         - 解析爬取到HTML数据
            - etree.HTML(page_text)
    - 使用etree对象中的xpath方法结合着不同xpath表达式实现标签定位和数据提取

#### xpath表达式
 - 标签定位
    - 最左侧的/:必须从根标签开始逐层定位
    - 非最左侧的/:表示一个层级
    - 最左侧的//:从任意位置定义目标标签
    - 非最左侧的//:表示多个层级
    - 属性定位：//tagNmae[@ attrName='value']
    - 索引定位：//tagNmae[index],index的索引是从1开始的
    - 模糊匹配：
        - //div[contains(@class,'ng')]定位到class属性值中包含ng的div标签
        - // div[starts-with(@class,'ta']定位到class属性值中是一ta开头的div标签
 - 数据提取
    - 取标签中的数据
        - /text():直系文本内容
        - //text():所有的文本内容
    - 取属性的数据
        - tagName/@attrName

In [30]:
from  lxml import etree

In [31]:
tree = etree.parse(r'./test.html')
tree

In [32]:
tree.xpath('/html/head')#从head标签开始定位
tree.xpath('//head')#将html文档中的所有的head标签定位到
tree.xpath('/html//meta')

[]

In [33]:
tree.xpath('//div[ @ class="song"]')

[<Element div at 0x20afa9ee3c0>]

需求：爬取https://pic.netbian.com/4kmeinv/中的图片

In [15]:

import requests
import os
from  lxml import etree
from tqdm import tqdm, trange
header={
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}
def url(index):
    url_list=list()
    url_list.append('https://pic.netbian.com/4kmeinv/')
    for i in range(1,index):
        url_list.append('https://pic.netbian.com/4kmeinv/index_{}.html'.format(i))
    return url_list
for url in tqdm(url(10), desc='It\'s a test'):
# for url in:
    page=requests.get(url=url,headers=header)
    page.encoding='gbk'
    page_text=page.text
    # print(page_text)
    tree=etree.HTML(page_text)
    #获取到页面开始定位标签
    li_list=tree.xpath('//div[@class="slist"]/ul/li')
    for li in li_list:
        img_src='https://pic.netbian.com/'+li.xpath('./a/img/@src')[0]
        img_name=li.xpath('./a/img/@alt')[0]+'.jpg'
        img_data=requests.get(img_src,headers=header).content
        file_path=os.path.join('/home/sxl/python_dev/python_study/crawler/study/part_first/crawler_data_analysis/img_meinv',img_name)
        with open(file_path,'wb')as fp:
            fp.write(img_data)
        print('{}，下载完成'.format(img_name))

It's a test:   0%|                                                                                                                                                                          | 0/10 [00:00<?, ?it/s]

清新 小仙女 绿色裙子 绿色丝带 养眼美腿 4k高清美女壁纸.jpg，下载完成
厨房 居家 可爱美女 短发 白色袜子 4k壁纸.jpg，下载完成
教室 JK制服美女 黑色丝袜 4k壁纸.jpg，下载完成
海琴烟 同人 cosplay美女4K高清壁纸.jpg，下载完成
绿色植物 清新养美胸美女4k壁纸3840x2160.jpg，下载完成
cosplay美女 电脑桌 键盘 机房 可爱 小姐姐4k壁纸3840x2160.jpg，下载完成
高颜值气质美女刘亦菲电脑壁纸4k超清全屏.jpg，下载完成
刘亦菲 牛仔衣 小花裙子 去有风的地方剧照 4k美女壁纸.jpg，下载完成
张钧甯 银光裙子 4k美女壁纸3840x2160.jpg，下载完成
刘亦菲 牛仔衣 小花裙子 去有风的地方剧照 3440x1440带鱼屏壁纸.jpg，下载完成
高山 户外 长发清纯美女4k壁纸.jpg，下载完成
居家 甜美美女 长发 粉色毛衣 4k美女壁纸.jpg，下载完成
长月烬明 白鹿剧照 4k美女壁纸.jpg，下载完成
刘亦菲 长卷发 高颜值气质美女 3440x1440带鱼屏壁纸.jpg，下载完成
长月烬明 白鹿古装剧照 4k美女电脑壁纸.jpg，下载完成
民族风美女 树 美女风景4k壁纸.jpg，下载完成
长月烬明 陈都灵剧照高清4k电脑壁纸.jpg，下载完成
青春 可爱 女生 闭眼 百褶裙 白色裤袜 4k美女壁纸.jpg，下载完成
刘亦菲 头纱 黑色裙子礼服 6k美女高清壁纸.jpg，下载完成


It's a test:  10%|████████████████▏                                                                                                                                                 | 1/10 [00:06<00:54,  6.10s/it]

清纯美女 白色 简单 居家 4k壁纸.jpg，下载完成


It's a test:  20%|████████████████████████████████▍                                                                                                                                 | 2/10 [00:06<00:21,  2.63s/it]

...

可爱 居家 女孩 画画 美女 4k 电脑壁纸.jpg，下载完成
家居 可爱小姐姐 侧躺 好看身材4k美女壁纸.jpg，下载完成
古风 居士 黑白 美女 壁纸 4k.jpg，下载完成
小清新 清纯 美女 裙子 鲜花 树林 摄影 4k 电脑 壁纸 3840x2160.jpg，下载完成
4k 周洁琼 木屋 鲜花 美女 壁纸 3840x2160.jpg，下载完成
4k 元气 少女 摩托车 帅气 美女 电脑 壁纸.jpg，下载完成
cosplay 原神 刻晴 小姐姐 白衬衫 黑色包臀裙 黑丝袜 4k美女壁纸.jpg，下载完成
居家 沙发 好看衣服 短裙 好看身材美女美腿4k壁纸3840x2160.jpg，下载完成
马凡舒 白色裙子 美女 4k 高清 壁纸.jpg，下载完成
白色毛衣 长发 美女 潘白雪 4k 电脑 壁纸.jpg，下载完成
4k 长发 居家 清纯美女 高清壁纸.jpg，下载完成
白鹿 红色古装美女3440x1440 高清壁纸.jpg，下载完成
4k 古装 古风 唯美 仰头美女壁纸.jpg，下载完成
4k古风 纸伞 树林 简单 美女护眼壁纸.jpg，下载完成
可爱长发美女孔雪儿4k壁纸3840x2160.jpg，下载完成
孔雪儿 清纯护眼美女4k壁纸.jpg，下载完成
芭蕾舞美女 好看身材 美腿 唯美4k美女壁纸.jpg，下载完成
漂亮芭蕾舞美女4k壁纸.jpg，下载完成
3440x1440 带鱼屏 居家 小清新 阳光 美女壁纸.jpg，下载完成
cosplay阿狸 居家美女 兽耳 尾巴 4k美女电脑壁纸.jpg，下载完成


It's a test:  90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                | 9/10 [00:49<00:06,  6.16s/it]

中秋节 月亮 cos 嫦娥 美女唯美4k壁纸.jpg，下载完成
韩雪 蝴蝶 4k超高清美女壁纸.jpg，下载完成
古风 仙气美女 荷花塘4k壁纸.jpg，下载完成
大波浪长发美女张婕婕4k电脑壁纸.jpg，下载完成
漂亮气质美女高颜值美女张婕婕 4k壁纸.jpg，下载完成
可爱美女模特张婕婕4k壁纸.jpg，下载完成
伞 美女 王秋紫4k电脑壁纸.jpg，下载完成
韩雪 蝴蝶不知身是梦3440x1440带鱼屏壁纸.jpg，下载完成
虞书欣 苍兰诀4k电脑壁纸.jpg，下载完成
林允儿 长头发 白色衬衫美女4k电脑壁纸3840x2160.jpg，下载完成
林允儿 黑色裙子礼服美女4k壁纸.jpg，下载完成
漂亮新娘 红色裙子 古装古风美女4k壁纸.jpg，下载完成
cosplay 鬼刀 美女 红色裙子 饰品 4k美女壁纸3840x2160.jpg，下载完成
架子鼓 吉他 清纯可爱短发美女4k壁纸.jpg，下载完成
清纯美女回头 养眼好看美女4k壁纸3840x2160.jpg，下载完成
居家沙发 女孩 清纯美女4k壁纸3840x2160.jpg，下载完成
森林 美女 背影 好看美背美女4k壁纸3840x2160.jpg，下载完成
清纯美女 嘴唇 手指 护眼美女高清4k壁纸.jpg，下载完成
白色婚纱裙子 白色丝袜美腿美女4k壁纸.jpg，下载完成
杨幂 绿色裙子美女4k壁纸.jpg，下载完成


It's a test: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:55<00:00,  5.58s/it]

杨幂 健身美女4k壁纸.jpg，下载完成
